# CNN DOA with 1 degree resolution

In [1]:
from preprocessing import *
from training import rmse
from music import get_all_predictions
from training import create_model, evaluate_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
# Label resolution of classification
RESOLUTION = 1

# Number of samples to include while creating one ML feature
SAMPLES = 2048

# Determines the overlap of samples between consecutive features
STEP = 1024

### Create training and testing sets

In [20]:
df_train = create_dataframe('train', samples=SAMPLES, step=STEP, resolution=RESOLUTION)
print()
df_test = create_dataframe('test', samples=SAMPLES, step=STEP, resolution=RESOLUTION)
print()

df_train.to_csv('../training_data/super_azimuth_train_dataset.csv')
df_test.to_csv('../training_data/super_azimuth_test_dataset.csv')

# Create numpy arrays with observations and one-hot labels
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train, y_train, X_test, y_test = create_whole_dataset(df_train, df_test, encoder)

np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test)

train file 3240/3240
test file 3240/3240


((628560, 15, 13), (210740, 15, 13), (628560, 360), (210740, 360))

Only run this when all the variables are not stored in memory (i.e. after restarting the kernel):

In [3]:
df_train = pd.read_csv('../training_data/super_azimuth_train_dataset.csv', index_col=[0])
df_test = pd.read_csv('../training_data/super_azimuth_test_dataset.csv', index_col=[0])
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit([[label] for label in df_train['label']])
X_train, y_train, X_test, y_test = create_whole_dataset(df_train, df_test, encoder)
np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test)

((628560, 15, 13), (210740, 15, 13), (628560, 360), (210740, 360))

### Fit and evaluate model

In [4]:
# Transpose the observations because Conv1D requires timesteps as the 1st dim
if X_train.shape[1] == MIC_COMBS:
    X_train, X_test = np.transpose(X_train, axes=[0, 2, 1]), np.transpose(X_test, axes=[0, 2, 1])
X_train.shape, X_test.shape

((628560, 13, 15), (210740, 13, 15))

In [5]:
model, history = create_model(X_train, y_train, X_test, y_test)
np.save('../models/super_history.npy', history.history)

Epoch 1/20
19643/19643 [==============================] - 40s 2ms/step - loss: 1.7252 - accuracy: 0.3354
Epoch 2/20
19643/19643 [==============================] - 42s 2ms/step - loss: 1.1309 - accuracy: 0.5108
Epoch 3/20
19643/19643 [==============================] - 44s 2ms/step - loss: 1.0003 - accuracy: 0.5721
Epoch 4/20
19643/19643 [==============================] - 44s 2ms/step - loss: 0.9336 - accuracy: 0.6050
Epoch 5/20
19643/19643 [==============================] - 46s 2ms/step - loss: 0.8943 - accuracy: 0.6257
Epoch 6/20
19643/19643 [==============================] - 46s 2ms/step - loss: 0.8667 - accuracy: 0.6399
Epoch 7/20
19643/19643 [==============================] - 47s 2ms/step - loss: 0.8488 - accuracy: 0.6509
Epoch 8/20
19643/19643 [==============================] - 47s 2ms/step - loss: 0.8317 - accuracy: 0.6596
Epoch 9/20
19643/19643 [==============================] - 47s 2ms/step - loss: 0.8174 - accuracy: 0.6671
Epoch 10/20
19643/19643 [==============================

In [15]:
# Test model
accuracy = evaluate_model(model, X_test, y_test)
print(f'Accuracy: {accuracy}')
y_pred_nn = encoder.inverse_transform(model.predict(X_test))
y_true_nn = encoder.inverse_transform(y_test)
print(f'RMSE: {rmse(y_true_nn, y_pred_nn)}')

6586/6586 [==============================] - 8s 1ms/step - loss: 0.9844 - accuracy: 0.7454
Accuracy: 0.745
RMSE: 29.946


In [16]:
def evaluate_for_property(df_train, df_test, prop, value):
    """
    Measures the model prediction for test samples
    with a given property, such as room size.
    """
    
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    # Filter test set by property value
    X_trn, y_trn, X_tst, y_tst = create_whole_dataset(
        df_train, df_test[df_test[prop]==value], encoder
    )
    
    # Evaluate the model on the filtered set
    X_tst = np.transpose(X_tst, axes=[0, 2, 1])
    loss, acc = model.evaluate(X_tst, y_tst, batch_size=batch_size, verbose=0)
    
    return round(loss, 3), round(acc, 3)


# Evaluate performance for different properties
print('Room sizes')
for room in ROOMS:
    _, acc = evaluate_for_property(df_train, df_test, 'room', room)
    print(f"{room} room accuracy: {acc}")
    
print('\nDistances')
for dist in np.unique(df_test.dist):
    _, acc = evaluate_for_property(df_train, df_test, 'dist', dist)
    print(f"{dist} cm distance accuracy: {acc}")

Room sizes
small room accuracy: 0.74
medium room accuracy: 0.743
large room accuracy: 0.753

Distances
50 cm distance accuracy: 0.419
150 cm distance accuracy: 0.919
200 cm distance accuracy: 0.927
250 cm distance accuracy: 0.908
350 cm distance accuracy: 0.886
450 cm distance accuracy: 0.895


In [35]:
model.save("../models/super_model")

INFO:tensorflow:Assets written to: ../models/super_model\assets


### Compare to MUSIC baseline

In [19]:
y_true, y_pred, info = get_all_predictions(True, samples=SAMPLES, step=STEP, resolution=RESOLUTION)
print()
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {round(accuracy, 3)}')
print(f'RMSE: {rmse(y_true, y_pred)}')

File 3240/3240
Accuracy: 0.29
RMSE: 2.183


In [20]:
def get_entries_with_property(info, prop, value):
    if prop == 'distance': i = 0
    elif prop == 'room': i = 1
        
    info = info[:, i]
    return np.where(info == value)

# Evaluate performance for different properties
print('Room sizes')
for room in ROOMS:
    indices = get_entries_with_property(info, 'room', room)
    y_true_room, y_pred_room = np.take(y_true, indices)[0], np.take(y_pred, indices)[0]
    accuracy = accuracy_score(y_true_room, y_pred_room)
    print(f"{room} room accuracy: {round(accuracy, 3)}")
    
print('\nDistances')
for dist in np.unique(info[:, 0]):
    indices = get_entries_with_property(info, 'distance', dist)
    y_true_dist, y_pred_dist = np.take(y_true, indices)[0], np.take(y_pred, indices)[0]
    accuracy = accuracy_score(y_true_dist, y_pred_dist)
    print(f"{dist} cm distance accuracy: {round(accuracy, 3)}")

Room sizes
small room accuracy: 0.232
medium room accuracy: 0.279
large room accuracy: 0.359

Distances
150 cm distance accuracy: 0.158
200 cm distance accuracy: 0.27
250 cm distance accuracy: 0.246
350 cm distance accuracy: 0.224
450 cm distance accuracy: 0.291
50 cm distance accuracy: 0.394
